# Cows per capita (per state)

On a [recent episode of 3 Minute Review](https://podcasters.spotify.com/pod/show/3minreview/episodes/Day-139-Alaska-Milch-e29esui), Mike wondered if Alaska had the highest cow-to-person ratio of any state (owing to there being relatively few people in Alaska).

Here, we try to answer that question.

In [1]:
import os
import requests

from dotenv import load_dotenv
load_dotenv()

True

## 1. Number of cows

The USDA has an API for data from the [National Agriculture Statistics Service](https://quickstats.nass.usda.gov/api/)

In [2]:
USDA_API_KEY = os.environ.get('USDA_API_KEY')

In [3]:
USDA_URL = f"https://quickstats.nass.usda.gov/api/api_GET/?key={USDA_API_KEY}&source_desc=SURVEY&year=2021&commodity_desc=CATTLE&short_desc=CATTLE, INCL CALVES - INVENTORY&statisticcat_desc=INVENTORY&domain_desc=TOTAL&agg_level_desc=STATE&format=JSON"

In [4]:
cattle_req = requests.get(USDA_URL)
cattle = [[x['Value'], x['location_desc'], x['state_fips_code']] for x in cattle_req.json()['data']]

In [5]:
cattle

[['1,295,000', 'ALABAMA', '01'],
 ['17,000', 'ALASKA', '02'],
 ['980,000', 'ARIZONA', '04'],
 ['1,780,000', 'ARKANSAS', '05'],
 ['5,150,000', 'CALIFORNIA', '06'],
 ['2,700,000', 'COLORADO', '08'],
 ['47,000', 'CONNECTICUT', '09'],
 ['13,000', 'DELAWARE', '10'],
 ['1,700,000', 'FLORIDA', '12'],
 ['1,090,000', 'GEORGIA', '13'],
 ['142,000', 'HAWAII', '15'],
 ['2,500,000', 'IDAHO', '16'],
 ['1,050,000', 'ILLINOIS', '17'],
 ['850,000', 'INDIANA', '18'],
 ['3,700,000', 'IOWA', '19'],
 ['6,550,000', 'KANSAS', '20'],
 ['2,090,000', 'KENTUCKY', '21'],
 ['775,000', 'LOUISIANA', '22'],
 ['76,000', 'MAINE', '23'],
 ['177,000', 'MARYLAND', '24'],
 ['37,000', 'MASSACHUSETTS', '25'],
 ['1,140,000', 'MICHIGAN', '26'],
 ['2,150,000', 'MINNESOTA', '27'],
 ['920,000', 'MISSISSIPPI', '28'],
 ['4,310,000', 'MISSOURI', '29'],
 ['2,450,000', 'MONTANA', '30'],
 ['6,850,000', 'NEBRASKA', '31'],
 ['470,000', 'NEVADA', '32'],
 ['32,000', 'NEW HAMPSHIRE', '33'],
 ['25,000', 'NEW JERSEY', '34'],
 ['1,390,000', 'N

## Population

The Census Bureau has an API for its ACS data. We used 2021 data for cows, so we want 2021 data for population.

In [6]:
CENSUS_URL = "https://api.census.gov/data/2021/acs/acs5/spt?get=B01001_001E,NAME&for=state:*"

In [7]:
pop_req = requests.get(CENSUS_URL)

In [8]:
pop = pop_req.json()

In [9]:
pop

[['B01001_001E', 'NAME', 'state'],
 ['4997675', 'Alabama', '01'],
 ['735951', 'Alaska', '02'],
 ['7079203', 'Arizona', '04'],
 ['6751340', 'Indiana', '18'],
 ['3006309', 'Arkansas', '05'],
 ['39455353', 'California', '06'],
 ['5723176', 'Colorado', '08'],
 ['3605330', 'Connecticut', '09'],
 ['981892', 'Delaware', '10'],
 ['683154', 'District of Columbia', '11'],
 ['21339762', 'Florida', '12'],
 ['10625615', 'Georgia', '13'],
 ['1453498', 'Hawaii', '15'],
 ['1811617', 'Idaho', '16'],
 ['12821813', 'Illinois', '17'],
 ['3179090', 'Iowa', '19'],
 ['2932099', 'Kansas', '20'],
 ['4494141', 'Kentucky', '21'],
 ['6141534', 'Missouri', '29'],
 ['6148545', 'Maryland', '24'],
 ['4657305', 'Louisiana', '22'],
 ['1357046', 'Maine', '23'],
 ['6991852', 'Massachusetts', '25'],
 ['10062512', 'Michigan', '26'],
 ['5670472', 'Minnesota', '27'],
 ['2967023', 'Mississippi', '28'],
 ['1077978', 'Montana', '30'],
 ['1951480', 'Nebraska', '31'],
 ['3059238', 'Nevada', '32'],
 ['1372175', 'New Hampshire', '3

## Combine the two

In [10]:
data_by_fips = {}

In [11]:
for row in cattle:
    count = int(row[0].replace(',',''))
    state = row[1].title()
    fips = row[2]
    data_by_fips[fips] = { 'state': state, 'cattle': count }

In [12]:
for row in pop:
    fips = row[2]
    if fips in data_by_fips:
        data_by_fips[fips]['population'] = int(row[0])

In [13]:
data_by_fips

{'01': {'state': 'Alabama', 'cattle': 1295000, 'population': 4997675},
 '02': {'state': 'Alaska', 'cattle': 17000, 'population': 735951},
 '04': {'state': 'Arizona', 'cattle': 980000, 'population': 7079203},
 '05': {'state': 'Arkansas', 'cattle': 1780000, 'population': 3006309},
 '06': {'state': 'California', 'cattle': 5150000, 'population': 39455353},
 '08': {'state': 'Colorado', 'cattle': 2700000, 'population': 5723176},
 '09': {'state': 'Connecticut', 'cattle': 47000, 'population': 3605330},
 '10': {'state': 'Delaware', 'cattle': 13000, 'population': 981892},
 '12': {'state': 'Florida', 'cattle': 1700000, 'population': 21339762},
 '13': {'state': 'Georgia', 'cattle': 1090000, 'population': 10625615},
 '15': {'state': 'Hawaii', 'cattle': 142000, 'population': 1453498},
 '16': {'state': 'Idaho', 'cattle': 2500000, 'population': 1811617},
 '17': {'state': 'Illinois', 'cattle': 1050000, 'population': 12821813},
 '18': {'state': 'Indiana', 'cattle': 850000, 'population': 6751340},
 '19':

## Calculate cattle-per-person and order the data

In [14]:
# Convert to list of dicts
data_list = [x for x in data_by_fips.values()]

In [15]:
# Calculate ratio of cows to people
for state in data_list:
    state['ratio'] = state['cattle'] / state['population']

In [16]:
# Sort by ratio
data_list = sorted(data_list, key=lambda i: (i['ratio']), reverse=True)

In [18]:
for index, item in enumerate(data_list, 1):
    print(f"{index}. {item['state']}: {item['ratio']:.2f} cows/person")

1. South Dakota: 4.54 cows/person
2. Nebraska: 3.51 cows/person
3. North Dakota: 2.52 cows/person
4. Montana: 2.27 cows/person
5. Wyoming: 2.25 cows/person
6. Kansas: 2.23 cows/person
7. Idaho: 1.38 cows/person
8. Oklahoma: 1.34 cows/person
9. Iowa: 1.16 cows/person
10. Missouri: 0.70 cows/person
11. New Mexico: 0.66 cows/person
12. Arkansas: 0.59 cows/person
13. Wisconsin: 0.59 cows/person
14. Colorado: 0.47 cows/person
15. Kentucky: 0.47 cows/person
16. Texas: 0.45 cows/person
17. Vermont: 0.38 cows/person
18. Minnesota: 0.38 cows/person
19. Mississippi: 0.31 cows/person
20. Oregon: 0.30 cows/person
21. Tennessee: 0.26 cows/person
22. Alabama: 0.26 cows/person
23. Utah: 0.25 cows/person
24. West Virginia: 0.21 cows/person
25. Louisiana: 0.17 cows/person
26. Virginia: 0.16 cows/person
27. Nevada: 0.15 cows/person
28. Washington: 0.15 cows/person
29. Arizona: 0.14 cows/person
30. California: 0.13 cows/person
31. Indiana: 0.13 cows/person
32. Michigan: 0.11 cows/person
33. Pennsylvania: